In [1]:
import joblib
import pandas as pd
import sqlite3 as sq

from itertools import chain

from src.db import (
    execute_fetchall,
    get_schema,
    get_schemas,
    get_schemas,
    drop_table,
    get_tables,
    count_tables,
    get_table,
    update_db
)

from src.format import (
    format_anime_base,
    format_X,
    format_anime_X,
    format_anime_title,
    format_anime
)

In [2]:
db = 'animedb.sqlite'

In [3]:
df_anime                = get_table('anime', db)
df_anime_title          = get_table('anime_title', db)
df_character            = get_table('character', db)
df_voiceactor           = get_table('voiceactor', db)
df_staff                = get_table('staff', db)
df_anime_character      = get_table('anime_character', db)
df_character_voiceactor = get_table('character_voiceactor', db)
df_anime_staff          = get_table('anime_staff', db)


In [4]:
df_anime_f = df_anime.merge(
    df_anime_title[df_anime_title['type'] == 'Default'][['anime_id', 'title']]
)[['anime_id', 'title', 'image_webp', 'score', 'scored_by', 'popularity']]

In [ ]:
df_anime['id']      = ('A_' + df_anime['anime_id'].astype(str))
df_character['id']  = ('C_' + df_character['character_id'].astype(str))
df_voiceactor['id'] = ('V_' + df_voiceactor['voiceactor_id'].astype(str))
df_staff['id']      = ('S_' + df_staff['staff_id'].astype(str))

In [5]:
import networkx as nx
G = nx.Graph()

G.add_nodes_from(pd.concat([
    ('A_' + df_anime['anime_id'].astype(str)),
    ('C_' + df_character['character_id'].astype(str)),
    ('V_' + df_voiceactor['voiceactor_id'].astype(str)),
    ('S_' + df_staff['staff_id'].astype(str))
]).values)

In [6]:
G.add_edges_from(pd.concat([
    ('A_' + df_anime_character['anime_id'].astype(str)),
    ('C_' + df_anime_character['character_id'].astype(str))
], axis=1).values)

G.add_edges_from(pd.concat([
    ('C_' + df_character_voiceactor['character_id'].astype(str)),
    ('V_' + df_character_voiceactor['voiceactor_id'].astype(str))
], axis=1).values)

G.add_edges_from(pd.concat([
    ('A_' + df_anime_staff['anime_id'].astype(str)),
    ('S_' + df_anime_staff['staff_id'].astype(str))
], axis=1).values)


In [10]:
[(s, [t for t in G.neighbors(s) if t.startswith('A')]) for s in G.neighbors('V_1')]

In [100]:
def unformat(s):
    if s.startswith('A'):
        return df_anime_f[df_anime_f['anime_id'] == int(s[2:])][['title']].assign(type='anime').values
    elif s.startswith('C'):
        return df_character[df_character['character_id'] == int(s[2:])][['name']].assign(type='character').values
    elif s.startswith('V'):
        return df_voiceactor[df_voiceactor['voiceactor_id'] == int(s[2:])][['name']].assign(type='voiceactor').values
    else:
        return df_staff[df_staff['staff_id'] == int(s[2:])][['name']].assign(type='staff').values

In [118]:
paths = [p for p in nx.all_shortest_paths(G, 'V_12961', 'V_1')]

In [119]:
paths

In [120]:
import numpy as np
[pd.concat([pd.DataFrame(unformat(n)) for n in p]) for p in paths]

In [89]:
H = G.subgraph(['V_12961', 'V_34785'])

In [37]:
df_anime_filter_nsw = df_anime\
    [~df_anime['rating'].isin(['R+ - Mild Nudity', 'Rx - Hentai'])]

In [43]:
df_anime_filter_nsw[['anime_id', 'image_webp', ]]